In [99]:
import pandas as pd
import numpy as np
import re
from natasha import ( # docs: https://github.com/natasha/natasha
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

from collections import Counter
from multiprocessing import Process
import os
import shutil
import time

In [100]:
test = pd.read_csv('datasets/test.csv', index_col=0)
train = pd.read_csv('datasets/train.csv', index_col=0)

## Просмотр комментариев собранных из разных источников

In [3]:
list(test[test['from']=='twitter'][:100]['text'].items())

[(206612,
  'Только что на паре по праву пели ним российской федерации )) ахха нам норм'),
 (123384,
  '@lama_popolama это, вроде, чувак из "мёртвые до востребования" - крутой сериал)))'),
 (138700, '@VekshinaI обожаю тебя и наши покатушки на машинке:*'),
 (115038, 'RT @make_me_strong_: @smille_Tommo пхаххаха:D Заипись'),
 (190170,
  '@zlyerusskie я вас поняли с английским полный порядок. т.е. вы се6я рабами что ли считаете?))'),
 (168469, '06.12.2013 / Вечерний Маскарад:) http://t.co/oYSJpAawmX'),
 (182375,
  '@wylsacom андройды привозят, реплики. Знакомая так купила айфон за 8 тыщ))'),
 (180203,
  '@KatePatrik \nтолько проснулась, лежу во мраке заточенья... :D\nИ слышу как бурлит живот. Нужно поесть немедля, но вставать лень :с'),
 (171545,
  '@mudrak_ylia ну вслучае свадьбы, куда мне слать приглашение ты знаешь))))'),
 (193063,
  'RT @gihomatu: Я тут почитал почту - смотрю фсе такие умные ;). Не фсе. Я глупый Ж;-). Кто еще за такую зарплату будет делать такую работу.'),
 (225299,
  

In [6]:
# Особенности: 
# RT @make_me_strong, o_O, :DDDDDD, http://t.co/bc5HNVjJAq', 100500, ахахахха ууууу, Анна Якубаб «Глорию Скотт»

In [5]:
list(test[test['from']=='otzovik'][:100]['text'].items())

[(26190,
  'Сейчас учусь на 3-ем курсе, и уже могу сделать вывод о данном ВУЗе. Самым главным плюсом является то, что поступить на бюджет можно и с достаточно низкими баллами(у меня было 234). \n\nНаходится ВУЗ конечно не в центре города, но само расположение достаточно благоприятное. Рядом находится набережная с панорамой на р. Волгу. Внешне он выглядит не очень, особенно 1-й корпус, но внутри все иначе, ремонт на высоком уровне. Аудитории не обшарпанные, в хорошем состоянии. Все преподаватели заслуженные специалисты в своем деле, в основном понимающие. Из минусов можно отметить постоянные перебежки между корпусами на переменах, так как пары иногда находятся в разных местах. Учиться придется, ВУЗ является одним из лучших юридических ВУЗов страны, хорошо котируется среди других. Сам город обычный, ничего сверхъестественного нет. Центр простой, небольшой, особо нигде не погуляешь. Грязноват. Из минусов можно отметь крайне большое количество магазинов с алкоголем, между ними дистанция ин

In [7]:
# Особенности: 
# Санкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3
# )В. А. Рождественский  Школа №321

In [8]:
list(test[test['from']=='vk'][:100]['text'].items())

[(14745, 'Пхахахахах!!!)) Ашооот!))'),
 (5881, 'Красивых много, а ЛЮБЛЮ Я ТЕБЯ'),
 (10353, 'Класно гульнули ;******'),
 (1173, 'Крестник любимый))'),
 (18616,
  'Что это у тебя в руке?\n- Счастье...\n- А почему такое маленькое?\n- Оно только моё. Зато какое\nлучистое и красивое...\n- Да, восхитительно!\n- Хочешь кусочек?\n- Наверное...\n- Давай ладошку, я поделюсь с\nтобой.\n- Ой, оно такое тёплое!\n- Нравится?\n- Очень...Спасибо! Знаешь, мне\nнамного лучше, когда счастье в\nруке...\n- Так всегда бывает.\n- А если я с кем-то поделюсь?\n- Тогда у тебя прибавится\nсвоего!\n- Почему?\n- Не знаю. Только потом оно\nстанет ещё более тёплым.\n- А руки об него обжечь\nможно?\n- Руки обжигает боль...Счастье\nне может обжечь...\nПоделисьсвоим кусочком\nсчастья со всеми, кому ты его\nжелаешь! И неважно,сколько\nвернется, ведь счастье не\nпродается, передавай\nвсем,кто,по-твоему , заслужил\nсчастья!\nДЕЛЮСЬ!!!!!!!!!!!!'),
 (10069,
  'Вернулась, наконец. Довольна, счастлива, отдохнувша. Родная комн

In [9]:
# Особенности: 
# Пхахахахах!!!)) , #затмение#2015#красота, @ Люблино Район Чуд, #умнымбытьмодно, 😎😃😃 @ Loropark Tenerife

## Очистка текста

In [10]:
example = '''
RT @make_me_strong, o_O, :DDDDDD, http://t.co/bc5HNVjJAq', 100500, ахахахха ууууу, Анна Якубаб «Глорию Скотт»
@VekshinaI обожаю тебя и наши покатушки на машинке:*
RT @make_me_strong_: @smille_Tommo пхаххаха:D Заипись
06.12.2013 / Вечерний Маскарад:) http://t.co/oYSJpAawmX
Санкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3
)В. А. Рождественский  Школа №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись...
Закончил академию BARBER WANTED 15.02.2019 года, "Школа123 _ 6"
Pre-Intermediate. Курс с носителем языка.
учебных центров г. Челябинска и выбрал Феникс.
С уважением, Кашин Сергей Сергеевич.
Ездил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможность обучаться в Китае! Менеджеры всегда на связи и готовы помочь разобраться с вопросами! Всем советую обращаться в компанию Red Planet.')
лечащего врача [ФИО вырезано модератором] и заведующую отделением [ФИО вырезано модератором], которые буквально вернули меня к жизни
Пхахахахах!!!)) -> хаха , #затмение#2015#красота, @ Люблино Район Чуд, #умнымбытьмодно, 😎😃😃 @ Loropark Tenerife
Челсиииииииииииииииии!!))) ЧЕмпионы))) Класно гульнули ;****** Счастье...\n- А почему такое маленькое?\n- Оно
аиста репачок дааааааааааа) проект Дьюса из HU) все классные, отношения Весов и Львицы и не идеальны
ТЕТЯ Ванюши (3500/50см)!!!=) Забавно :) Тебе моя ласточка😘P.S слушать👂исключительно☝️в🚘
какое...............а....
такой губастый*?)хаха
герой русских комиксов=) слово.неслово ы1в
'''

In [11]:
def clear_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text) # Удаление ссылок
    text = re.sub('@[\S_]+', '', text) # Удаление username
    text = re.sub('".+"', '', text) # Удаление названий. Пример: "ООО ..."
    text = re.sub('«.+»', '', text) 
    text = re.sub('#.+', '', text) # Удаление хештегов
    # text = text.lower().replace("ё", "е")

    for i in list('?!()') + ['...', '[ФИО вырезано модератором]', 'RT']: # .
        text = text.replace(i, ' ') # В конце строки часто не ставят пробелов
        # из-за этого после обработки два слова склеиваются и перестают нести смысловую нагрузку для сети
    text = text.replace('\\n', ' . ')
    
    # symbols = re.findall("[а-я\s]", text) # Оставляем только буквы
    # text = ''.join(symbols)
    # text = re.sub(' +', ' ', text) # Оставляем между словами 1 пробел
    return text.strip()

In [13]:
clear_text(example)

"o_O, :DDDDDD,  100500, ахахахха ууууу, Анна Якубаб \n обожаю тебя и наши покатушки на машинке:*\n    пхаххаха:D Заипись\n06.12.2013 / Вечерний Маскарад:  \nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3\n В. А. Рождественский  Школа №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись \nЗакончил академию BARBER WANTED 15.02.2019 года, \nPre-Intermediate. Курс с носителем языка.\nучебных центров г. Челябинска и выбрал Феникс.\nС уважением, Кашин Сергей Сергеевич.\nЕздил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас  студентов  встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможность обучаться в Китае  Менеджеры всегда на связи и готовы помочь разобраться с вопросами  Всем советую обращаться в компанию Red Planet.' \nле

In [14]:
re.findall('#[\S\d]+', example)

['#затмение#2015#красота,', '#умнымбытьмодно,']

In [15]:
t = 'Пхахахахах!!!)) Ашооот!)) 	аххахахахаххххахха сахара'
re.sub(r'(хаха)+', ' хаха ', t)   

'П хаха х!!!)) Ашооот!)) \tах хаха ххххахха сахара'

In [16]:
text = '''аккорд, Челсиииииииииииииииии 
дааааааааааа
ууууу
аббат, аббревиатура, аккорд, аккордеон, аккуратность, аккуратный, аксессуар, аллея, аллигатор, антенна, бриллиант, брокколи, грамм, грамматика, граммофон
ахахахха'''

r = re.search(r'(.)\1+\s', text) # (.)\1+\s

while r:
    print(r)
    span = r.span()
    text = text[: span[0]+1] + ' ' + text[span[1]:]

    r = re.search(r'(.)\1+\s', text)


print(text)

<re.Match object; span=(12, 30), match='иииииииииииииииии '>
<re.Match object; span=(16, 28), match='ааааааааааа\n'>
<re.Match object; span=(18, 24), match='ууууу\n'>
аккорд, Челси 
да у аббат, аббревиатура, аккорд, аккордеон, аккуратность, аккуратный, аксессуар, аллея, аллигатор, антенна, бриллиант, брокколи, грамм, грамматика, граммофон
ахахахха


## Lib natasha

In [17]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
doc = Doc(example)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

for span in doc.spans:
    span.normalize(morph_vocab)

In [18]:
doc.spans

[DocSpan(start=1, stop=19, type='ORG', text='RT @make_me_strong', tokens=[...], normal='RT @make_me_strong'),
 DocSpan(start=84, stop=95, type='PER', text='Анна Якубаб', tokens=[...], normal='Анна Якубаб'),
 DocSpan(start=112, stop=121, type='ORG', text='VekshinaI', tokens=[...], normal='VekshinaI'),
 DocSpan(start=164, stop=166, type='ORG', text='RT', tokens=[...], normal='RT'),
 DocSpan(start=186, stop=198, type='ORG', text='smille_Tommo', tokens=[...], normal='smille_Tommo'),
 DocSpan(start=274, stop=293, type='LOC', text='Санкт-Петербургская', tokens=[...], normal='Санкт-Петербургская'),
 DocSpan(start=332, stop=341, type='ORG', text='Baby time', tokens=[...], normal='Baby time'),
 DocSpan(start=352, stop=358, type='PER', text='Анхель', tokens=[...], normal='Анхель'),
 DocSpan(start=361, stop=366, type='PER', text='Хорхе', tokens=[...], normal='Хорхе'),
 DocSpan(start=394, stop=421, type='PER', text='В. А. Рождественский  Школа', tokens=[...], normal='В. А. Рождественский  Школа'),

In [21]:
# Удаляем названия организаций, локаций и людей
remove = [(word.text, word.start, word.stop) for word in doc.spans]
print(remove)
t1 = example[:]
result = ''
start = 0

for i in remove:
    result+=t1[start: i[1]]
    start=i[2]
result

[('RT @make_me_strong', 1, 19), ('Анна Якубаб', 84, 95), ('VekshinaI', 112, 121), ('RT', 164, 166), ('smille_Tommo', 186, 198), ('Санкт-Петербургская', 274, 293), ('Baby time', 332, 341), ('Анхель', 352, 358), ('Хорхе', 361, 366), ('В. А. Рождественский  Школа', 394, 421), ('BARBER WANTED', 495, 508), ('Челябинска', 602, 612), ('Феникс', 622, 628), ('Кашин Сергей Сергеевич', 643, 665), ('Китай', 675, 680), ('Red Planet', 693, 703), ('ВУЗе', 772, 776), ('Китае', 797, 802), ('Китай', 817, 822), ('Влад', 917, 921), ('Игорь', 924, 929), ('Китае', 999, 1004), ('Red Planet', 1108, 1118), ('Пхахахахах', 1255, 1265), ('Люблино Район Чуд', 1307, 1324), ('Дьюса', 1510, 1515), ('HU', 1519, 1521), ('Весов', 1547, 1552), ('Львицы', 1555, 1561)]


'\n, o_O, :DDDDDD, http://t.co/bc5HNVjJAq\', 100500, ахахахха ууууу,  «Глорию Скотт»\n@ обожаю тебя и наши покатушки на машинке:*\n @make_me_strong_: @ пхаххаха:D Заипись\n06.12.2013 / Вечерний Маскарад:) http://t.co/oYSJpAawmX\n, 9:00 - 12:15 день, О центре развития , .Спасибо  и , 7.10.2020 года, конкурс3\n) №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись...\nЗакончил академию  15.02.2019 года, "Школа123 _ 6"\nPre-Intermediate. Курс с носителем языка.\nучебных центров г.  и выбрал .\nС уважением, .\nЕздил в  от компании  в 2019 году. Помогли собрать весь пакет документов, с оформлением в  и с обустройством в . По приезду в , нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие.  и  молодцы, благодарю их за предоставленную мне возможность обучаться в ! Менеджеры всегда на связи и готовы помочь разобраться с вопросами! Всем советую обращаться в компанию .\')\nлечащего врача [ФИО вырезано модератором] и заведующую отделением [ФИО вырезано мод

In [22]:
doc.tokens

[DocToken(start=1, stop=3, text='RT', id='1_1', head_id='1_0', rel='root', pos='X', feats=<Yes>),
 DocToken(start=4, stop=5, text='@', id='1_2', head_id='1_1', rel='flat:foreign', pos='X', feats=<Yes>),
 DocToken(start=5, stop=19, text='make_me_strong', id='1_3', head_id='1_1', rel='appos', pos='X', feats=<Yes>),
 DocToken(start=19, stop=20, text=',', id='1_4', head_id='1_5', rel='punct', pos='PUNCT'),
 DocToken(start=21, stop=24, text='o_O', id='1_5', head_id='1_8', rel='conj', pos='PROPN', feats=<Yes>),
 DocToken(start=24, stop=25, text=',', id='1_6', head_id='1_82', rel='punct', pos='PUNCT'),
 DocToken(start=26, stop=27, text=':', id='1_7', head_id='1_8', rel='punct', pos='PUNCT'),
 DocToken(start=27, stop=33, text='DDDDDD', id='1_8', head_id='1_52', rel='appos', pos='PROPN', feats=<Yes>),
 DocToken(start=33, stop=34, text=',', id='1_9', head_id='1_72', rel='punct', pos='PUNCT'),
 DocToken(start=35, stop=39, text='http', id='1_10', head_id='1_8', rel='conj', pos='X', feats=<Yes>),
 

In [23]:
doc.sents

[DocSent(start=1, stop=440, text="RT @make_me_strong, o_O, :DDDDDD, http://t.co/bc5..., tokens=[...], spans=[...]),
 DocSent(start=443, stop=476, text='Мы уже не ждали и не надеялись...', tokens=[...]),
 DocSent(start=477, stop=558, text='Закончил академию BARBER WANTED 15.02.2019 года, ..., tokens=[...], spans=[...]),
 DocSent(start=559, stop=629, text='Курс с носителем языка.\nучебных центров г. Челяб..., tokens=[...], spans=[...]),
 DocSent(start=630, stop=666, text='С уважением, Кашин Сергей Сергеевич.', tokens=[...], spans=[...]),
 DocSent(start=667, stop=716, text='Ездил в Китай от компании Red Planet в 2019 году...., tokens=[...], spans=[...]),
 DocSent(start=717, stop=803, text='Помогли собрать весь пакет документов, с оформлен..., tokens=[...], spans=[...]),
 DocSent(start=804, stop=916, text='По приезду в Китай, нас (студентов) встретил сотр..., tokens=[...], spans=[...]),
 DocSent(start=917, stop=1005, text='Влад и Игорь молодцы, благодарю их за предоставле..., tokens=[...],

In [24]:
doc.sents[5].syntax.print()

     Ездил    
  ┌► в        case
  └─ Китай    
  ┌► от       case
┌─└─ компании 
└►┌─ Red      appos
  └► Planet   flat:foreign
┌──► в        case
│ ┌► 2019     amod
└─└─ году     
     .        


## Определяем важность слов с хештегами

In [25]:
all_texts = ' '.join(test['text'].to_list() + train['text'].to_list())

In [26]:
all_hashtags = re.findall('#[\S\d]+', all_texts)
hashtags = set(all_hashtags)

In [27]:
hashtags

{'#вкусный',
 '#ZeeRabbit',
 '#документы',
 '#sportfaza',
 '#burgerking',
 '#MyChemicalRomanceWeMiss',
 '#премьера',
 '#euromaidan',
 '#proactivefm',
 '#танцыдоутра,',
 '#твитдождю',
 '#nailru…',
 '#GetWellSoonSuga',
 '#моисамыелюбимыеродители!#люблю!',
 '#печальбеда',
 '#good…',
 '#отецисын',
 '#ntvplus',
 '#personalway',
 '#автоворонки',
 '#цитаты',
 '#Рубин',
 '#жизньпрекрасна',
 '#едудомой…',
 '#sad',
 '#anekdot',
 '#ЗинаШамоян',
 '#Голос',
 '#RussiaLoves1D',
 '#МеждународныйДеньСпасибо',
 '#Форсаж7',
 '#пятница',
 '#жиробас',
 '#ессентуки',
 '#самоубийство',
 '#t54',
 '#умничать',
 '#minecarft',
 '#crazy',
 '#Доброе_утро!!!',
 '#послероссии',
 '#доверие#любимыйразрешает😘😍🚘',
 '#гаррипоттер',
 '#треш',
 '#весело',
 '#happiness',
 '#baikonur',
 '#school2',
 '#интернетмаркетинг',
 '#євромай…',
 '#art',
 '#smile',
 '#друзьязаботушки',
 '#Волгограде',
 '#просто',
 '#TWDFinale',
 '#Восхитительнаятренировка',
 '#флюрография',
 '#thailand',
 '#навальный',
 '#Воспоминания2013Года',
 '#заеб

In [28]:
len(all_hashtags), len(hashtags)

(1693, 1255)

In [29]:
hashtag_counter = Counter(all_hashtags)

In [30]:
[i for i in hashtag_counter.most_common() if i[1] != 1]

[('#євромайдан', 41),
 ('#RT', 41),
 ('#TEAMFOLLOWBACK', 18),
 ('#ПредновогодниеОбнимашки', 17),
 ('#mtvstars', 16),
 ('#AutoFollowBack', 14),
 ('#ХочуПровестиНовогоднююНочьВместеС', 14),
 ('#FOLLOW', 14),
 ('#FF', 11),
 ('#евромайдан', 11),
 ('#FF_RU', 11),
 ('#follow', 10),
 ('#RU_FF', 10),
 ('#ЧитаюВзаимно', 8),
 ('#биатлон', 8),
 ('#Голос', 7),
 ('#ff', 7),
 ('#ВзаимныйФолловинг', 6),
 ('#Молодежка', 6),
 ('#euromaidan', 6),
 ('#ДИРЕКШИОНЕРФОЛЛОВЬДИРЕКШИОНЕРА', 6),
 ('#курс_МагияЖенскогоБелья', 6),
 ('#HappyBirthdayIanFromRussia', 5),
 ('#rufollowback', 5),
 ('#love', 5),
 ('#followme', 5),
 ('#like', 5),
 ('#nice', 5),
 ('#2', 5),
 ('#TeamFollowBack', 5),
 ('#followback', 4),
 ('#sbsum', 4),
 ('#radiot', 4),
 ('#Снаступающимтвиттерский', 4),
 ('#SLEDUI', 4),
 ('#жизньболь', 4),
 ('#четверг', 4),
 ('#.', 4),
 ('#girl', 4),
 ('#webcamtoy', 4),
 ('#…', 4),
 ('#EMIотзыв', 4),
 ('#np', 3),
 ('#статусы', 3),
 ('#счастье', 3),
 ('#Євромайдан', 3),
 ('#Волгоград', 3),
 ('#читаювзаимно', 3

## Лемматизация

In [33]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)

In [37]:
[(i.text, i.lemma) for i in doc.tokens[110:125]]

[('Мы', 'мы'),
 ('уже', 'уже'),
 ('не', 'не'),
 ('ждали', 'ждать'),
 ('и', 'и'),
 ('не', 'не'),
 ('надеялись', 'надеяться'),
 ('...', '...'),
 ('Закончил', 'закончить'),
 ('академию', 'академия'),
 ('BARBER', 'barber'),
 ('WANTED', 'wanted'),
 ('15.02.2019', '15.02.2019'),
 ('года', 'год'),
 (',', ',')]

In [38]:
def remove_words(text):
    text = text.replace('[ФИО вырезано модератором]', ' ')
    text = text.replace('RT', ' ')
    
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text) # Удаление ссылок
    text = re.sub('@[\S_]+', '', text) # Удаление username
    text = re.sub('".+"', '', text) # Удаление названий. Пример: "ООО ..."
    text = re.sub('«.+»', '', text) 
    text = re.sub('#[\S\d_]+', '', text) # Удаление хештегов
    text = text.replace("ё", "е")
    text = text.replace('\\n', ' . ')
    text = re.sub(r'(хаха)+', ' ха_ха ', text)   
    return text

In [41]:
tmp = remove_words(example)
tmp

"\n   o_O, :DDDDDD,  100500, а ха_ха хха ууууу, Анна Якубаб \n обожаю тебя и наши покатушки на машинке:*\n    пхах ха_ха :D Заипись\n06.12.2013 / Вечерний Маскарад:) \nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3\n)В. А. Рождественский  Школа №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись...\nЗакончил академию BARBER WANTED 15.02.2019 года, \nPre-Intermediate. Курс с носителем языка.\nучебных центров г. Челябинска и выбрал Феникс.\nС уважением, Кашин Сергей Сергеевич.\nЕздил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможность обучаться в Китае! Менеджеры всегда на связи и готовы помочь разобраться с вопросами! Всем советую обращаться в компанию Red 

In [39]:
duplicates = re.compile(r'(.)\1{2,}')

def remove_duplicates(text):
    dupl = duplicates.search(text)
    
    while dupl:
        span = dupl.span()
        text = text[: span[0]+1] + text[span[1]:]
        dupl = duplicates.search(text)
    del dupl
    return text

In [42]:
tmp = remove_duplicates(tmp)
tmp

"\n o_O, :D,  100500, а ха_ха хха у, Анна Якубаб \n обожаю тебя и наши покатушки на машинке:*\n пхах ха_ха :D Заипись\n06.12.2013 / Вечерний Маскарад:) \nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3\n)В. А. Рождественский  Школа №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись.\nЗакончил академию BARBER WANTED 15.02.2019 года, \nPre-Intermediate. Курс с носителем языка.\nучебных центров г. Челябинска и выбрал Феникс.\nС уважением, Кашин Сергей Сергеевич.\nЕздил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможность обучаться в Китае! Менеджеры всегда на связи и готовы помочь разобраться с вопросами! Всем советую обращаться в компанию Red Planet.')\nлечащ

In [43]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

def create_doc(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    return doc, morph_vocab

def remove_names_organizations(text):
    doc, morph_vocab = create_doc(text)
    
    for span in doc.spans:
        span.normalize(morph_vocab) # detect only NER objects

    remove = [(word.text, word.start, word.stop) for word in doc.spans]
    result = ''
    start = 0
    
    for i in remove:
        result+=text[start: i[1]]
        start=i[2]
    else:
        result+=text[start:]
    
    if len(remove) == 0:
        result = text

    del doc, morph_vocab
    return result

In [47]:
create_doc(tmp)
tmp = remove_names_organizations(tmp)
tmp

"\n o_O, :D,  100500, а ха_ха хха у,  \n обожаю тебя и наши покатушки на машинке:*\n пхах ха_ха :D Заипись\n06.12.2013 / Вечерний Маскарад:) \nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития , .Спасибо  и , 7.10.2020 года, конкурс3\n) №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись.\nЗакончил академию  15.02.2019 года, \n. Курс с носителем языка.\nучебных центров г.  и выбрал .\nС уважением, .\nЕздил в  от компании  в 2019 году. Помогли собрать весь пакет документов, с оформлением в  и с обустройством в . По приезду в , нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие.  и  молодцы, благодарю их за предоставленную мне возможность обучаться в ! Менеджеры всегда на связи и готовы помочь разобраться с вопросами! Всем советую обращаться в компанию .')\nлечащего врача и заведующую отделением  , которые буквально вернули меня к жизни\nП ха_ха х!)) ->  ха_ха  ,  @ ,  😎😃😃 @ Loropark Tenerife\nЧелси!!) ЧЕмпионы) Класно гульнули ;* Счасть

In [45]:
def take_only_text(text):
    text = text.lower()
    for i in list('?!().-0123456789;:,#$%&\\/*+<=>@[') + ['...']: # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~
        text = text.replace(i, ' ')
    text = re.sub('[^\x00-\x7Fа-яА-Я]', ' ', text) # Удаляем эмоджи
    symbols = re.findall("[а-я\s_]", text) # Оставляем только буквы
    text = ''.join(symbols)
    text = re.sub(' +', ' ', text) # Оставляем между словами 1 пробел
    del symbols
    return text.strip()

In [48]:
tmp = take_only_text(tmp)
tmp

'_ а ха_ха хха у \n обожаю тебя и наши покатушки на машинке \n пхах ха_ха заипись\n вечерний маскарад \nсанкт петербургская день о центре развития спасибо и года конкурс \n нам года \n\nмы уже не ждали и не надеялись \nзакончил академию года \n курс с носителем языка \nучебных центров г и выбрал \nс уважением \nездил в от компании в году помогли собрать весь пакет документов с оформлением в и с обустройством в по приезду в нас студентов встретил сотрудник компании и координировал нашу группу по пути в общежитие и молодцы благодарю их за предоставленную мне возможность обучаться в менеджеры всегда на связи и готовы помочь разобраться с вопросами всем советую обращаться в компанию \nлечащего врача и заведующую отделением которые буквально вернули меня к жизни\nп ха_ха х ха_ха \nчелси чемпионы класно гульнули счастье \n а почему такое маленькое \n оно\nаиста репачок да проект из все классные отношения и и не идеальны\nтетя ванюши см забавно тебе моя ласточка слушать исключительно в \nкако

In [49]:
def lemmatization(text):
    doc, morph_vocab = create_doc(text)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    result = [i.lemma for i in doc.tokens]
    del doc, morph_vocab
    return result

In [50]:
tmp = lemmatization(tmp)
tmp

['_',
 'а',
 'ха_ха',
 'хха',
 'у',
 'обожать',
 'ты',
 'и',
 'наш',
 'покатушка',
 'на',
 'машинка',
 'пха',
 'ха_хий',
 'заипеться',
 'вечерний',
 'маскарад',
 'санкт',
 'петербургский',
 'день',
 'о',
 'центр',
 'развитие',
 'спасибо',
 'и',
 'год',
 'конкурс',
 'мы',
 'год',
 'мы',
 'уже',
 'не',
 'ждать',
 'и',
 'не',
 'надеяться',
 'закончить',
 'академия',
 'год',
 'курс',
 'с',
 'носитель',
 'язык',
 'учебный',
 'центр',
 'г',
 'и',
 'выбрать',
 'с',
 'уважение',
 'ездить',
 'в',
 'от',
 'компания',
 'в',
 'год',
 'помочь',
 'собрать',
 'весь',
 'пакет',
 'документ',
 'с',
 'оформление',
 'в',
 'и',
 'с',
 'обустройство',
 'в',
 'по',
 'приезд',
 'в',
 'мы',
 'студент',
 'встретить',
 'сотрудник',
 'компания',
 'и',
 'координировать',
 'наш',
 'группа',
 'по',
 'путь',
 'в',
 'общежитие',
 'и',
 'молодец',
 'благодарить',
 'они',
 'за',
 'предоставить',
 'я',
 'возможность',
 'обучаться',
 'в',
 'менеджер',
 'всегда',
 'на',
 'связь',
 'и',
 'готовый',
 'помочь',
 'разобраться'

In [51]:
def clear(text):
    funcs = [remove_words, remove_duplicates, remove_names_organizations,
             take_only_text, lemmatization]
    for func in funcs:
        # print(text)
        if len(text.strip()) != 0:
            text = func(str(text))
    del funcs
    return text

In [53]:
' '.join(clear(example))

'_ а ха_ха хха у обожать ты и наш покатушка на машинка пха ха_хий заипеться вечерний маскарад санкт петербургский день о центр развитие спасибо и год конкурс мы год мы уже не ждать и не надеяться закончить академия год курс с носитель язык учебный центр г и выбрать с уважение ездить в от компания в год помочь собрать весь пакет документ с оформление в и с обустройство в по приезд в мы студент встретить сотрудник компания и координировать наш группа по путь в общежитие и молодец благодарить они за предоставить я возможность обучаться в менеджер всегда на связь и готовый помочь разобраться с вопрос весь советовать обращаться в компания лечить врач и заведовать отделение который буквально вернуть я к жизнь п ха_ха х ха_хий челси чемпион класно гульнули счастие а почему такой маленький оно аист репачок да проект из весь классный отношение и и не идеальный тетя ванюша см забавно ты мой ласточка слушать исключительно в какой а такой губастый ха_ха герой русский комикс слово неслово ы в'

In [54]:
example

'\nRT @make_me_strong, o_O, :DDDDDD, http://t.co/bc5HNVjJAq\', 100500, ахахахха ууууу, Анна Якубаб «Глорию Скотт»\n@VekshinaI обожаю тебя и наши покатушки на машинке:*\nRT @make_me_strong_: @smille_Tommo пхаххаха:D Заипись\n06.12.2013 / Вечерний Маскарад:) http://t.co/oYSJpAawmX\nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3\n)В. А. Рождественский  Школа №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись...\nЗакончил академию BARBER WANTED 15.02.2019 года, "Школа123 _ 6"\nPre-Intermediate. Курс с носителем языка.\nучебных центров г. Челябинска и выбрал Феникс.\nС уважением, Кашин Сергей Сергеевич.\nЕздил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможност

## Emoji

In [56]:
re.sub('[^\x00-\x7Fа-яА-Я]', ' ', example)

'\nRT @make_me_strong, o_O, :DDDDDD, http://t.co/bc5HNVjJAq\', 100500, ахахахха ууууу, Анна Якубаб  Глорию Скотт \n@VekshinaI обожаю тебя и наши покатушки на машинке:*\nRT @make_me_strong_: @smille_Tommo пхаххаха:D Заипись\n06.12.2013 / Вечерний Маскарад:) http://t.co/oYSJpAawmX\nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3\n)В. А. Рождественский  Школа  321 нам 3.3 года. \n\nМы уже не ждали и не надеялись...\nЗакончил академию BARBER WANTED 15.02.2019 года, "Школа123 _ 6"\nPre-Intermediate. Курс с носителем языка.\nучебных центров г. Челябинска и выбрал Феникс.\nС уважением, Кашин Сергей Сергеевич.\nЕздил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможност

In [96]:
'😫😭❤😬'.encode('utf-8')

b'\xf0\x9f\x98\xab\xf0\x9f\x98\xad\xe2\x9d\xa4\xf0\x9f\x98\xac'

In [57]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

deEmojify(example)

'\nRT @make_me_strong, o_O, :DDDDDD, http://t.co/bc5HNVjJAq\', 100500, ахахахха ууууу, Анна Якубаб «Глорию Скотт»\n@VekshinaI обожаю тебя и наши покатушки на машинке:*\nRT @make_me_strong_: @smille_Tommo пхаххаха:D Заипись\n06.12.2013 / Вечерний Маскарад:) http://t.co/oYSJpAawmX\nСанкт-Петербургская, 9:00 - 12:15 день, О центре развития Baby time, .Спасибо Анхель и Хорхе, 7.10.2020 года, конкурс3\n)В. А. Рождественский  Школа №321 нам 3.3 года. \n\nМы уже не ждали и не надеялись...\nЗакончил академию BARBER WANTED 15.02.2019 года, "Школа123 _ 6"\nPre-Intermediate. Курс с носителем языка.\nучебных центров г. Челябинска и выбрал Феникс.\nС уважением, Кашин Сергей Сергеевич.\nЕздил в Китай от компании Red Planet в 2019 году. Помогли собрать весь пакет документов, с оформлением в ВУЗе и с обустройством в Китае. По приезду в Китай, нас (студентов) встретил сотрудник компании и координировал нашу группу по пути в общежитие. Влад и Игорь молодцы, благодарю их за предоставленную мне возможност

In [58]:
clear('СКУЧАЮ ПО ТЕБЕ АБЫЕМКА МОЙ ЛЮБИМЫЙ!!((** хахаха')

['скучать', 'по', 'ты', 'абыемкий', 'мой', 'любимый', 'ха_ха', 'ха']

In [59]:
clear(str(np.nan))

''

## Очистка датасета

In [60]:
def clear_row(row):
    row['lemma'] = ' '.join(clear(row['text']))
    return row

In [61]:
%%time
df = test[3295:3300].apply(clear_row, axis=1) # 3298

CPU times: user 93.1 ms, sys: 243 ms, total: 336 ms
Wall time: 28 ms


In [62]:
test[2400+720:3600-240]['text'].tolist()

['теперь и работать буду в воскресенье.............(',
 'Больше всего боюсь,что когда ты наладишь с ней общение,ты про меня забудешь:(',
 '"@koffboy: Каждый из нас надеется в глубине души, что ВКонтакте не заработает." К сожалению, уже заработал... Ээх, как это печально(',
 '@blogoped хреново то,что  внимательно читать и не надо! Хочешь-не хочешь,а содержание узнаешь!((((((((((((((((',
 'RT @e_lebedeva: А куда пропали обычные "советские" мандарины? Невозможно найти(( в каждом магазине лежит по пять разных сортов, но они "не …',
 'я тут подумала й...\nмені нічого втрачати, одже в мене нічого нема.\n:(',
 '@imtheallonsy аввввв, я б тебя поцеловала, но не могу у меня язык от укола онемел и не двигается :(',
 'RT @come_____true: @bad_meets__evil  ага\nЯ вообще в шоке o_O',
 'Легла в 9, проснулась в 2 и больше не смогла заснуть -_-\nВо-первых, опух глаз из-за ячменя\nВо-вторых, мне нельзя ложиться рано\nДома сегодня(',
 'Люди мне нужен попутчица на 13 декабря во вьетнам!;( может у кого то п

In [63]:
test.iloc[3298].tolist()

['negative',
 '@DHO_OKEAHA_ #Снаступающимтвиттерский http://t.co/uXagx3CMgH',
 'twitter',
 '   ']

In [64]:
clear('@DHO_OKEAHA_ #Снаступающимтвиттерский http://t.co/uXagx3CMgH')

'  '

In [68]:
try:
    create_doc([])
except Exception as e:
    print('Exception:', e)

Exception: expected string or bytes-like object


In [71]:
%%time
df = test.apply(clear_row, axis=1)

## Распараллеливание обработки

In [88]:
dir_name = '.ds_tmp'

try:
    os.mkdir(dir_name)
except:
    pass

In [77]:
def devide_texts(df, parts = 5):
    l = len(df)//parts
    data = []
    amount = 0
    
    for i in range(parts):
        if i + 1 == parts:
            data.append(df[i*l:])
        else:
            data.append(df[i*l:(i+1)*l])
            
        print('Length', len(data[i]), 'Start idx:',  i*l,  'End idx:',(i+1)*l)
        amount+=len(data[i])
    
    print('Amount:', amount, '/', len(df['text'].tolist()))
    for i, value in enumerate(data, start = 1):
       print(f'Part {i}, data len: {len(value)}')

    return data

In [81]:
tmp = devide_texts(test)
len(tmp)

Length 1200 Start idx: 0 End idx: 1200
Length 1200 Start idx: 1200 End idx: 2400
Length 1200 Start idx: 2400 End idx: 3600
Length 1200 Start idx: 3600 End idx: 4800
Length 1200 Start idx: 4800 End idx: 6000
Amount: 6000 / 6000
Part 1, data len: 1200
Part 2, data len: 1200
Part 3, data len: 1200
Part 4, data len: 1200
Part 5, data len: 1200


5

In [79]:
def worker(i, data):
    text = data.apply(clear_row, axis=1)
    text.to_csv(f'{dir_name}/clean_text_{i}.csv')

In [89]:
# %%time
# worker(0, tmp[0])

CPU times: user 32.2 s, sys: 1min 30s, total: 2min 2s
Wall time: 7.8 s


In [90]:
def join_results(parts = 5):
    data = []

    for i in range(parts):
        data.append(pd.read_csv(f'{dir_name}/clean_text_{i}.csv', index_col=0))
    
    return pd.concat(data)

In [93]:
def preprocess(data, num_workers = 5, ds_name='clean_text.csv'):
    thread_list = []

    try:
        os.mkdir(dir_name)
    except:
        print('Directory exists')
    start_time = time.time()

    for i in range(num_workers):
        thread_list.append(Process(target=worker, args=(i, data[i])))
        thread_list[i].start()

    for i in range(num_workers):
        thread_list[i].join()

    df = join_results(parts = num_workers)
    df.to_csv(ds_name)
    
    end_time = time.time()
    print(f'Elapsed time: {end_time-start_time} sec')
    print(f'Saved to {ds_name}')

    try:
        shutil.rmtree(dir_name)
    except:
        pass

In [94]:
df = pd.concat([train, test])
df.columns, df.shape

(Index(['label', 'text', 'from', 'lemma'], dtype='object'), (51555, 4))

In [101]:
# Test ds
data = devide_texts(test)
preprocess(data, ds_name='test_ds_clean_text.csv')

Length 1200 Start idx: 0 End idx: 1200
Length 1200 Start idx: 1200 End idx: 2400
Length 1200 Start idx: 2400 End idx: 3600
Length 1200 Start idx: 3600 End idx: 4800
Length 1200 Start idx: 4800 End idx: 6000
Amount: 6000 / 6000
Part 1, data len: 1200
Part 2, data len: 1200
Part 3, data len: 1200
Part 4, data len: 1200
Part 5, data len: 1200
Directory exists


Process Process-4:
Process Process-1:
Process Process-3:
Process Process-5:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/u

KeyboardInterrupt: 

  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/pandas/core/frame.py", line 10374, in apply
    return op.apply().__finalize__(self, method="apply")
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/pandas/core/apply.py", line 916, in apply
    return self.apply_standard()
  File "/tmp/ipykernel_42051/3557121087.py", line 2, in worker
    text = data.apply(clear_row, axis=1)
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/pandas/core/apply.py", line 916, in apply
    return self.apply_standard()
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/pandas/core/apply.py", line 1063, in apply_standard
    results, res_index = self.apply_series_generator()
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/pandas/core/frame.py", line 10374, in apply
    return op.apply().__finalize__(self, method="apply")
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/pandas/core/frame.py", line 1

In [102]:
# Train ds
data = devide_texts(train)
preprocess(data, ds_name='train_ds_clean_text.csv')

Length 9111 Start idx: 0 End idx: 9111
Length 9111 Start idx: 9111 End idx: 18222
Length 9111 Start idx: 18222 End idx: 27333
Length 9111 Start idx: 27333 End idx: 36444
Length 9111 Start idx: 36444 End idx: 45555
Amount: 45555 / 45555
Part 1, data len: 9111
Part 2, data len: 9111
Part 3, data len: 9111
Part 4, data len: 9111
Part 5, data len: 9111
Directory exists


Process Process-10:
Process Process-7:
Process Process-8:
Traceback (most recent call last):
Process Process-6:
Traceback (most recent call last):
Process Process-9:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/

KeyboardInterrupt: 

  File "/tmp/ipykernel_42051/784287105.py", line 2, in clear_row
    row['lemma'] = ' '.join(clear(row['text']))
  File "/tmp/ipykernel_42051/3439879933.py", line 7, in clear
    text = func(str(text))
  File "/tmp/ipykernel_42051/784287105.py", line 2, in clear_row
    row['lemma'] = ' '.join(clear(row['text']))
  File "/tmp/ipykernel_42051/4055807752.py", line 13, in create_doc
    doc.parse_syntax(syntax_parser)
  File "/tmp/ipykernel_42051/4055807752.py", line 13, in create_doc
    doc.parse_syntax(syntax_parser)
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/natasha/syntax.py", line 79, in map
    for markup in markups:
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/natasha/doc.py", line 139, in parse_syntax
    parse_syntax_doc(self, parser)
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/site-packages/natasha/doc.py", line 139, in parse_syntax
    parse_syntax_doc(self, parser)
  File "/home/den/Documents/UNIC/venv3/lib/python3.10/